In [1]:
import sys
import pathlib
import numpy as np
import pandas as pd
import optuna
import torch
import logging 

from optimize_utils import get_optimize_args, objective

script_directory = pathlib.Path("../utils/").resolve()
sys.path.insert(0, str(script_directory))
from data_loader import load_train_test_data

In [2]:
# Load command line arguments
args = get_optimize_args()


# Load data
data_directory = pathlib.Path("../0.data-download/data").resolve()

train_data, test_data, val_data, load_gene_stats = load_train_test_data(
    data_directory, train_or_test="all", load_gene_stats=True, zero_one_normalize=True
)


In [3]:
# Convert dataframes to tensors
train_tensor = torch.tensor(train_data, dtype=torch.float32)
test_tensor = torch.tensor(test_data, dtype=torch.float32)
val_tensor = torch.tensor(val_data, dtype=torch.float32)

In [4]:
# Run Optuna optimization and save study
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "BetaVAE-Optimization"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)
study = optuna.create_study(study_name=study_name, storage=storage_name, direction="minimize", load_if_exists=True)
study.optimize(
    lambda trial: objective(trial, train_tensor, val_tensor, train_data), n_trials=500
)

[I 2024-07-22 14:11:42,269] A new study created in RDB with name: BetaVAE-Optimization


A new study created in RDB with name: BetaVAE-Optimization
Epoch 0, Loss: 394.06888409514926
Epoch 1, Loss: 292.2269152285448
Epoch 2, Loss: 247.3566056436567
Epoch 3, Loss: 215.48063636893656
Epoch 4, Loss: 192.7217729710821
Epoch 5, Loss: 172.58360541044777
Epoch 6, Loss: 155.8281833022388
Epoch 7, Loss: 143.90983558768656
Epoch 8, Loss: 132.97709305037313
Epoch 9, Loss: 122.77803463152985
Epoch 10, Loss: 114.04519808185634
Epoch 11, Loss: 106.6701266616138
Epoch 12, Loss: 99.90931742653918
Epoch 13, Loss: 93.36831783465485
Epoch 14, Loss: 87.76339493936567
Epoch 15, Loss: 82.31756427821828
Epoch 16, Loss: 78.2445604011194
Epoch 17, Loss: 74.44000699626865
Epoch 18, Loss: 70.80466928055037
Epoch 19, Loss: 67.9257426247668
Epoch 20, Loss: 65.08383570429105
Epoch 21, Loss: 62.31833241021455
Epoch 22, Loss: 60.64309956564832
Epoch 23, Loss: 58.95850483908582
Epoch 24, Loss: 57.54225636951959
Epoch 25, Loss: 56.209088590251866
Epoch 26, Loss: 55.10607400011661
Epoch 27, Loss: 53.86453256

[I 2024-07-22 14:11:44,252] Trial 0 finished with value: 103.37539164225261 and parameters: {'latent_dim': 68, 'beta': 7.22427289288623, 'learning_rate': 0.0038669995249842977, 'batch_size': 67, 'epochs': 41}. Best is trial 0 with value: 103.37539164225261.


Epoch 35, Loss: 50.24153779442631
Epoch 36, Loss: 49.87954866779384
Epoch 37, Loss: 49.63312150186567
Epoch 38, Loss: 49.33465084246735
Epoch 39, Loss: 49.574412240555034
Epoch 40, Loss: 49.33133198752332
Trial 0 finished with value: 103.37539164225261 and parameters: {'latent_dim': 68, 'beta': 7.22427289288623, 'learning_rate': 0.0038669995249842977, 'batch_size': 67, 'epochs': 41}. Best is trial 0 with value: 103.37539164225261.
Epoch 0, Loss: 412.8970178404851
Epoch 1, Loss: 346.3121181203358
Epoch 2, Loss: 312.5519079407649
Epoch 3, Loss: 289.9535983413013
Epoch 4, Loss: 273.398923594916
Epoch 5, Loss: 259.87216505363807
Epoch 6, Loss: 250.75629700618003
Epoch 7, Loss: 243.96525259444962
Epoch 8, Loss: 235.67915731401587
Epoch 9, Loss: 229.43653582672576
Epoch 10, Loss: 223.9367624766791
Epoch 11, Loss: 218.92367836112408
Epoch 12, Loss: 214.57913217117536
Epoch 13, Loss: 210.4358740963153
Epoch 14, Loss: 205.7446485832556
Epoch 15, Loss: 201.92706827192166
Epoch 16, Loss: 198.8922

[I 2024-07-22 14:11:45,202] Trial 1 finished with value: 239.32786009046765 and parameters: {'latent_dim': 49, 'beta': 8.979389117535526, 'learning_rate': 0.0005069535188824445, 'batch_size': 47, 'epochs': 25}. Best is trial 0 with value: 103.37539164225261.


...

Epoch 441, Loss: 47.85824539412314
Epoch 442, Loss: 47.93336555139342
Trial 41 finished with value: 100.36641099717882 and parameters: {'latent_dim': 10, 'beta': 1.4047062598340716, 'learning_rate': 0.0025590968994500043, 'batch_size': 25, 'epochs': 443}. Best is trial 14 with value: 99.39484575059679.
Epoch 0, Loss: 206.23097033144822
Epoch 1, Loss: 182.65727466184703
Epoch 2, Loss: 162.7959079116138
Epoch 3, Loss: 145.52366178142492
Epoch 4, Loss: 129.94748243645057
Epoch 5, Loss: 114.99124291263409
Epoch 6, Loss: 102.47012137797341
Epoch 7, Loss: 91.85809863645639
Epoch 8, Loss: 82.62440741240088
Epoch 9, Loss: 74.86320390843633
Epoch 10, Loss: 68.8255883971257
Epoch 11, Loss: 63.8490260793202
Epoch 12, Loss: 60.29065451550839
Epoch 13, Loss: 57.2522729674382
Epoch 14, Loss: 54.97331160360308
Epoch 15, Loss: 53.25787613142782
Epoch 16, Loss: 52.14195971133104
Epoch 17, Loss: 51.03724948256763
Epoch 18, Loss: 50.30012343677122
Epoch 19, Loss: 49.75449400944496
Epoch 20, Loss: 49.2816

[I 2024-07-22 14:23:29,965] Trial 42 finished with value: 99.17800415886774 and parameters: {'latent_dim': 15, 'beta': 1.0040911324462307, 'learning_rate': 0.0020293966706959464, 'batch_size': 20, 'epochs': 310}. Best is trial 42 with value: 99.17800415886774.


Epoch 308, Loss: 47.6263355767549
Epoch 309, Loss: 47.51067199707031
Trial 42 finished with value: 99.17800415886774 and parameters: {'latent_dim': 15, 'beta': 1.0040911324462307, 'learning_rate': 0.0020293966706959464, 'batch_size': 20, 'epochs': 310}. Best is trial 42 with value: 99.17800415886774.
Epoch 0, Loss: 205.51000685051307
Epoch 1, Loss: 181.69901068388526
Epoch 2, Loss: 161.72682413129664
Epoch 3, Loss: 142.37382520988805
Epoch 4, Loss: 127.2362791146805
Epoch 5, Loss: 113.1272364374417
Epoch 6, Loss: 100.65731729535914
Epoch 7, Loss: 89.77115232553055
Epoch 8, Loss: 81.11940726664528
Epoch 9, Loss: 73.48532259357509
Epoch 10, Loss: 67.35338690458839
Epoch 11, Loss: 63.230183433418844
Epoch 12, Loss: 58.998908472772854
Epoch 13, Loss: 56.2752587161847
Epoch 14, Loss: 54.27597328299907
Epoch 15, Loss: 52.852012793697526
Epoch 16, Loss: 51.470990901206854
Epoch 17, Loss: 50.761857491108906
Epoch 18, Loss: 49.98824954815765
Epoch 19, Loss: 49.58927849157533
Epoch 20, Loss: 49.

[I 2024-07-22 14:23:47,199] Trial 43 finished with value: 99.7019280327691 and parameters: {'latent_dim': 18, 'beta': 1.0398381437452375, 'learning_rate': 0.00208104483887923, 'batch_size': 20, 'epochs': 315}. Best is trial 42 with value: 99.17800415886774.


Epoch 312, Loss: 47.75795169659515
Epoch 313, Loss: 47.52266508643307
Epoch 314, Loss: 47.704196485832554
Trial 43 finished with value: 99.7019280327691 and parameters: {'latent_dim': 18, 'beta': 1.0398381437452375, 'learning_rate': 0.00208104483887923, 'batch_size': 20, 'epochs': 315}. Best is trial 42 with value: 99.17800415886774.
Epoch 0, Loss: 199.36089195137592
Epoch 1, Loss: 147.00379656891326
Epoch 2, Loss: 109.53186171802122
Epoch 3, Loss: 83.11179071682602
Epoch 4, Loss: 67.53055210398205
Epoch 5, Loss: 58.30923616779384
Epoch 6, Loss: 53.59762964960355
Epoch 7, Loss: 51.15429846920184
Epoch 8, Loss: 49.92833593567806
Epoch 9, Loss: 49.28773284456623
Epoch 10, Loss: 48.987219420475746
Epoch 11, Loss: 48.763478065604595
Epoch 12, Loss: 48.697552034748135
Epoch 13, Loss: 48.477774184497434
Epoch 14, Loss: 48.51288365606052
Epoch 15, Loss: 48.37885359579058
Epoch 16, Loss: 48.4216575508687
Epoch 17, Loss: 48.3971727058069
Epoch 18, Loss: 48.45919736036614
Epoch 19, Loss: 48.2907

In [ ]:
# Save best hyperparameters
best_trial = study.best_trial
print(best_trial)
print(f"Best trial: {best_trial.values}")
print(f"Best hyperparameters: {best_trial.params}")